# Deploying our RAG app with Anyscale + Ray

In this notebook, we will deploy our RAG app using Ray Serve. Ray Serve is a scalable and programmable model serving library built on Ray. It is designed to simplify the process of serving and deploying models at scale.

## Develop a Serve app locally

The fastest way to develop a Ray Serve app is locally within the workspace. A Serve app running within a workspace behaves identically to a Serve app running as a production service, only it does not have a stable DNS name or fault tolerance.

To get started, let's view file the called `main.py` which performs the following additions

```python
from fastapi import FastAPI
from fastapi.responses import StreamingResponse
from ray import serve

... # same code as before

app = FastAPI()

@serve.deployment
@serve.ingress(app)
class QA:

    # FastAPI will automatically parse the HTTP request for us.
    @app.get("/answer")
    def answer(
        self,
        query: str,
        top_k: int,
        include_sources: bool = True,
    ):
        return StreamingResponse(...)
```

### Run the app locally
Run the command below to run the serve app locally on `localhost:8000`.

If you want to deploy again, just run the command again to update the deployment.

**Tip**: to more easily view Serve backend logs, you may find it convenient to use `serve run main:my_app --blocking` in a new VSCode terminal. This will block and print out application logs (exceptions, etc.) in the terminal.

In [ ]:
!serve run main:my_app --non-blocking

### Send a test request

Run the following cell to query the local serve app.

In [ ]:
import requests

params = dict(
    query="How can I deploy Ray Serve to Kubernetes?",
    top_k=3,
)

response = requests.get(f"http://localhost:8000/answer", params=params)
for chunk in response.iter_content(chunk_size=None, decode_unicode=True):
    print(chunk.decode(), end="")

## Deploy to production as a service

In order to enable fault tolerance and expose your app to the public internet, you must "Deploy" the application, which will create an Anyscale Service backed by a public load balancer. 

This service will run in a separate Ray cluster from the workspace, and will be monitored by the Anyscale control plane to recover on node failures. You will also be able to deploy rolling updates to the service without incurring downtime.

Use the following command to deploy your app as `my_service`.

In [ ]:
!serve deploy main:my_app --name=my_service